In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # setting ignore as a parameter and further adding category
from whispr import *
import os


In [2]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 3
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/231121_gly_shunt/'
sp_plasmids_file = folder + 'genex_sp_231121.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0, engine='openpyxl')
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + 'genex_pl_384.csv'
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + 'genex_mt_231121.xlsx'
#mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)
mt_genex = pd.read_excel(mt_genex_file, index_col = 0, dtype = str, engine = 'openpyxl').fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'plasmids_sp_updated.xlsx')
protocol_genex_df = protocol_genex_df[0]


Volumes used from each well for this protocol:
{'1_A1': 3.3, '1_P1': 0.35, '1_P2': 0.45, '1_P3': 0.1, '1_P5': 0.1, '1_P9': 0.05, '1_O17': 0.1, '1_M10': 0.125, '1_M11': 1.05, '1_M12': 0.375}


In [3]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


# change this#
txtl_well = 'E4'
##

for label, dwell in rxn_loc.items():
    for d in dwell:
        print(label)
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

pyc
neg
gly_shunt
pyc_gly_shunt
ms
icl
pyc_acs
sds
TXTL needed: 18.0


In [ ]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [ ]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [ ]:
folder = os.getcwd() + '/Experiments/230824/'


In [6]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + 'biosyn_sp_231121.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0, engine = 'openpyxl')
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
#sp_genex = sp_from_layout(layout_genex, 60)
layout_genex_384_file = folder + 'genex_pl_384.csv'
layout_genex_384 = pd.read_csv(layout_genex_384_file, index_col = 0, dtype = str)
sp_genex = sp_from_layout(layout_genex_384, 60)


# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + 'biosyn_mt_231121.xlsx'
mt_biosyn = pd.read_excel(mt_biosyn_file, index_col = 0, dtype = str, engine = 'openpyxl').fillna(0)
mt_biosyn = mt_biosyn[~mt_biosyn.index.isna()]

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + 'biosyn_pl.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
layout_biosyn1 = layout_biosyn1[~layout_biosyn1.index.isna()]

# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

#vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

## changed vol_table_df to change reaction volume ##
vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 25, total_vol = 25, fill_with='HEPES')


protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

Volumes used from each well for this protocol:
{'1_A1': 13.0, '1_B1': 13.0, '1_C1': 6.5, '1_D1': 8.5, '1_E2': 25.0, '1_E3': 5.0, '1_H1': 19.0, '1_H2': 7.0, '1_J1': 15.0, '1_K1': 22.0, '1_O1': 2.0, '1_P1': 4.0, '1_G4': 11.25, '1_C2': 7.7, '1_C6': 3.0, '2_A2': 693.05, '3_H9': 10.0, '3_H10': 35.0, '3_H11': 5.0, '3_H12': 5.0, '3_H13': 5.0, '3_H14': 5.0, '3_H15': 5.0, '3_H16': 5.0, '3_H17': 10.0, '3_H18': 10.0}


In [7]:
protocol_biosyn_dfs[0].to_csv(folder+'protocol_biosyn_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv(folder+'protocol_biosyn_6RES.csv',index = False)